In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

df = pd.read_csv('Downloads/MFTD.csv')

df['super_category'] = pd.cut(df['ATU'].str.extract('(\d+)').astype(int).squeeze(), # extract numbers and convert dtype to int
                              [-1, 299, 749, 849, 999, 1199, 1999, 2399],
                              labels=['ANIMAL TALES', # (-1, 299]
                                      'TALES OF MAGIC', # (299, 749]
                                      'RELIGIOUS TALES', # ...
                                      'REALISTIC TALES',
                                      'TALES OF THE STUPID OGRE (GIANT, DEVIL)',
                                      'ANECDOTES AND JOKES',
                                      'FORMULA TALES'])

df = df[~df['super_category'].isna()]

data = df['content'].to_list()

embeddings = model.encode(data, show_progress_bar=True)

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

In [17]:
X = embeddings

In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df["super_category"])

In [25]:
weights = sklearn.utils.class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)

class_weights = {k:v for k,v in enumerate(weights)}
class_weights

<IPython.core.display.Javascript object>

{0: 1.246031746031746,
 1: 0.5372112917023096,
 2: 5.356076759061834,
 3: 3.484049930651872,
 4: 3.518207282913165,
 5: 0.28847037207165827,
 6: 8.971428571428572}

In [19]:
from sklearn.utils import shuffle

X, y = shuffle(X, y)

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [32]:
len(np.unique(y))

7

In [33]:
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Dense(512, input_shape=X.shape[1:]),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(np.unique(y)), activation='softmax')
])

In [39]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

In [41]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
63/63 [==============================] - 0s 2ms/step - loss: 0.0424 - accuracy: 0.9935 - val_loss: 0.8196 - val_accuracy: 0.8056
Epoch 2/20
63/63 [==============================] - 0s 1ms/step - loss: 0.0319 - accuracy: 0.9970 - val_loss: 0.8365 - val_accuracy: 0.8095
Epoch 3/20
63/63 [==============================] - 0s 1ms/step - loss: 0.0283 - accuracy: 0.9980 - val_loss: 0.9379 - val_accuracy: 0.8056
Epoch 4/20
63/63 [==============================] - 0s 2ms/step - loss: 0.0216 - accuracy: 0.9990 - val_loss: 0.9402 - val_accuracy: 0.8214
Epoch 5/20
63/63 [==============================] - 0s 1ms/step - loss: 0.0223 - accuracy: 0.9965 - val_loss: 0.9510 - val_accuracy: 0.8016
Epoch 6/20
63/63 [==============================] - 0s 1ms/step - loss: 0.0168 - accuracy: 0.9990 - val_loss: 0.9590 - val_accuracy: 0.8214
Epoch 7/20
63/63 [==============================] - 0s 1ms/step - loss: 0.0133 - accuracy: 0.9995 - val_loss: 0.9993 - val_accuracy: 0.8214
Epoch 8/20
63/63 [==

In [42]:
model.evaluate(X_test, y_test)

8/8 [==============================] - 0s 1ms/step - loss: 1.2281 - accuracy: 0.8088


[1.2281465530395508, 0.8087649345397949]